# Using GAP Effectively: Some Tips and Pitfalls

Based on Steve Linton's talk at the Second CoDiMa Training School in Computational Discrete Mathematics (ICMS, Edinburgh, 17-21 October 2016, https://www.codima.ac.uk/school2016/)

## Philosophy

* GAP has two somewhat contradictory design goals
    * to allow users to pose questions in a way that seems natural to a working mathematician and get answers
    * to allow the expert computational mathematician to implement and apply the most advanced techniques to solve hard problems
* The first is achieved to a limited extent.

**Problem:** find an element of $S_9$ which is NOT an involution

In [128]:
n:=9;; Filtered(Elements(SymmetricGroup(n)), x-> x*x <> ())[1];

(7,8,9)

* Replace 9 by 10 and it is already visibly longer

In [130]:
n:=10;; Filtered(Elements(SymmetricGroup(n)), x-> x*x <> ())[1];

(8,9,10)

* Put `n:=15` and you quickly run out of memory: $15!$ is roughly $1.3 \times 10^{12}$

## This talk is about how to start thinking like an expert

## If you don't need it, then don't store it!
* What is happening in this command?
```
n:=9;; Filtered(Elements(SymmetricGroup(n)), x-> x*x <> ())[1];
```
    * First it computes and stores the full list of elements of $S_n$
    * Then it checks each of them to see if it has order dividing 2 and stores a second list of all of those which don’t
    * Finally it returns the first one

* Instead, we can stop looking when we find one. GAP even provides a built in function to do this:

In [138]:
n = 9;; First(Elements(SymmetricGroup(n)), x-> x*x <> ());

(8,9,10)

* Stopping things as soon as possible is an important principle
* In this case though the real problem is computing and storing all the elements of $S_n$
* Let's explore some other alternatives

## Enumerators

* Enumerator returns a list of elements of a domain which may be virtual
    * also EnumeratorSorted — but only if you need it
* For many objects it is quick to construct, but may be slower to access
* Do not remove the 2nd semicolon below yet (see https://github.com/gap-packages/JupyterKernel/issues/72)

In [28]:
e := Enumerator(SymmetricGroup(99));;

In [15]:
Length(e);

933262154439441526816992388562667004907159682643816214685929638952175999932299156089414639761565182862536979208272237582511852109168640000000000000000000000

In [17]:
e[10^100];

(2,60,99,55,54,65,7,16,18,32,70,15,5,37,43,97,19,31,66,30,90,17,29,85,28,67,27,62,26,34,52,59)(3,44,73,47,95,45,51,68,50,86,49,83,40,36,81,35,93,12,76,11,75,10,46,9,96,8,53,42,41,22,78,21,38,20,24,63,23,48,39,56,4,6,58,14,80,13,25,33)

See also `EnumeratorOfTuples` and `EnumeratorOfCombinations` (both documented) and `EnumeratorOfCartesianProduct` (currently undocumented)

In [142]:
?EnumeratorOfCombinations

Reference: EnumeratorOfCombinations

## Iterators

* Even an Enumerator can be too heavyweight
    * sometimes you don’t need to even number the elements, or know how many there are
* For this GAP has _Iterators_
    * `IsDoneIterator` and `NextIterator` operations

In [19]:
n := 9;; i := Iterator(SymmetricGroup(n));; 

In [20]:
while not IsDoneIterator(i) do x := NextIterator(i); if x*x = () then break; fi; od;

In [21]:
x;

()

Or more concisely, thanks to some built-in magic:

In [23]:
n := 9;; for x in SymmetricGroup(n) do if x *x = () then break; fi; od;

In [24]:
x;

()

Or even shorter:

In [30]:
n := 9;; First(SymmetricGroup(n), x->x*x = ());

()

## Randomness

* Sometimes you can’t even make an iterator for your group easily, but you know the elements you want exist and are not too rare
* So make pseudo-random elements of the group until you find one

In [31]:
g := SL(10,3);

SL(10,3)

In [32]:
repeat x := PseudoRandom(g); until Order(x) = (3^10-1)/2;

In [35]:
x;

[ [ Z(3)^0, Z(3), Z(3), Z(3)^0, Z(3)^0, Z(3), 0*Z(3), 0*Z(3), Z(3), Z(3) ], [ 0*Z(3), 0*Z(3), Z(3), Z(3)^0, Z(3)^0, Z(3), 0*Z(3), Z(3), 0*Z(3), Z(3) ], [ 0*Z(3), Z(3), Z(3), 0*Z(3), Z(3)^0, Z(3)^0, 0*Z(3), Z(3)^0, 0*Z(3), 0*Z(3) ], [ 0*Z(3), 0*Z(3), Z(3), Z(3), Z(3)^0, Z(3), Z(3)^0, Z(3)^0, 0*Z(3), Z(3)^0 ], [ 0*Z(3), Z(3)^0, 0*Z(3), 0*Z(3), 0*Z(3), 0*Z(3), Z(3)^0, 0*Z(3), Z(3), Z(3)^0 ], [ Z(3), Z(3), 0*Z(3), Z(3)^0, Z(3), 0*Z(3), Z(3), 0*Z(3), Z(3), Z(3)^0 ], [ Z(3), 0*Z(3), Z(3), 0*Z(3), Z(3), Z(3), Z(3)^0, Z(3)^0, 0*Z(3), Z(3) ], [ Z(3)^0, 0*Z(3), 0*Z(3), 0*Z(3), Z(3)^0, Z(3), 0*Z(3), Z(3)^0, 0*Z(3), 0*Z(3) ], [ Z(3), 0*Z(3), Z(3)^0, Z(3)^0, 0*Z(3), 0*Z(3), Z(3), 0*Z(3), 0*Z(3), Z(3) ], [ 0*Z(3), Z(3), 0*Z(3), Z(3)^0, Z(3), Z(3)^0, Z(3), Z(3), Z(3), Z(3)^0 ] ]

In [34]:
Display(x);

 1 2 2 1 1 2 . . 2 2
 . . 2 1 1 2 . 2 . 2
 . 2 2 . 1 1 . 1 . .
 . . 2 2 1 2 1 1 . 1
 . 1 . . . . 1 . 2 1
 2 2 . 1 2 . 2 . 2 1
 2 . 2 . 2 2 1 1 . 2
 1 . . . 1 2 . 1 . .
 2 . 1 1 . . 2 . . 2
 . 2 . 1 2 1 2 2 2 1


### But is searching through all the elements the right thing to do in the first place?
* Element order is a conjugacy invariant
* For many groups there are ways of finding conjugacy class representatives that are faster than listing all elements
    * or they might be already known and stored

In [39]:
n := 9;; Representative(First(ConjugacyClasses(SymmetricGroup(n)), c->Representative(c)^2 <> ()));

(1,2,3)

* This is one of the most powerful techniques, especially for non-abelian simple groups and things close to them
* Of course if you are really working in $S_n$ you can simply construct the answer as a permutation

## Narrowing the Search

In [40]:
First(SymmetricGroup(12), x-> OnTuples([1,2,3,4,5],x) = [1,3,5,7,9] and Order(x) = 7); 

(2,3,5,9,4,7,12)

* For values larger than 12, this gets slow
    * because it searches lots of elements that fix 2 before it looks at anything that moves 1 to 2
* Use a bit of maths
    * the elements that map [1,2,3,4,5] to [1,3,5,7,9] lie in a coset of a sequence stabilizer

In [42]:
g := SymmetricGroup(12);; s := Stabilizer(g,[1,2,3,4,5],OnTuples);;

In [43]:
r := RepresentativeAction(g,[1,2,3,4,5],[1,3,5,7,9],OnTuples);

(2,3,5,9,4,7)

In [44]:
First(s,x->Order(x*r) = 7)*r;

(2,3,5,9,12,4,7)

## General Principles

### Searching for an element in a group

* Don’t write down the list of elements first
* Stop when you’ve found it
* Stop looking at other elements as soon as you know they’re not it
    * order of a matrix can be large and a bit slow to compute
    * if all you care about is whether it is 2, just check `IsOne(x*x) and not IsOne(x)`
* Try to identify a subgroup, or coset or conjugacy class that it lies in
    * remember Sylow subgroups!
    * automorphism group sometimes helps too
* Search only in there

### Seaching for a subgroup

* Even worse — quite small groups can have very many subgroups
* Some kinds that are eas(ier) to find
    * Cyclic subgroups (via `ConjugacyClasses`)
    * `NormalSubgroups`
    * Derived, Lower Central etc. series
    * Sylow subgroups
    * Maximal subgroups (for some groups)
        * `MaximalSubgroups` will return all subgroups. You are likely to want ony `MaximalSubgroupClassReps`
* Ask yourself if one of these lists might include the one you want, or at least help you on your way

### Searching for multiple elements

**Conjecture:** $U_3(3)$ cannot be generated by three involutions

In [147]:
g := PSU(3,3);

<permutation group of size 6048 with 2 generators>

**So we know some things not to do:**
* list all 216G triples of elements of $U_3(3)$ and filter out all the ones that generate the group and consist of involutions
* use `IteratorOfTuples` to run through all 216G ...
* use `IteratorOfCombinations` to run through 36G of unordered triples
* the same, but test for involutions first
    * would take a few hours on a laptop
    
** What to do:**
* find the involutions first (there are just 63 of them) and run over triples

In [148]:
invs := Filtered(g, x->Order(x) = 2);;

In [149]:
Length(invs);

63

In [151]:
i := IteratorOfCombinations(invs,3); ct := 0;

<object>

0

In [152]:
while not IsDoneIterator(i) do
    x := NextIterator(i);
    if Subgroup(g,x) = g then 
        break; 
    fi; 
    ct := ct+1; 
od;

In [153]:
ct;

39711

In [154]:
Binomial(63,3);

39711

## Searching for multiple elements

* We still haven’t used conjugacy
* We could choose our first involution to be a conjugacy class representative
    * there is only one conjugacy class of involutions
    * reduce search from `Binomial(63,3)` to `Binomial(62,2)`
* But now the second involution can be chosen up to conjugacy in the centraliser of the first one
    * just four cases to consider
        * search is now $4 \times 61$ cases
* Of course the third one can be chosen up to conjugacy in the normaliser of the subgroup generated by the first two
* If the things you are searching for are not all the same, then the order in which you look at them also matters

## Morpheus

* This type of search for sequences of elements that generate something is nicely implemented by Alexander Hulpke in a part of the GAP library called **Morpheus**
* There are various functions that access morpheus documented in the library under "Searching for Homomorphisms"
* Our example is asking whether $U_3(3)$ is a quotient of the free product of three cyclic groups of order 2

In [57]:
g:=PSU(3,3);;

In [64]:
F:=FreeGroup(3);

<group with 3 generators>

In [65]:
F:=F/[F.1^2,F.2^2,F.3^2];

<group with 3 generators>

In [66]:
GQuotients(F,g);

[  ]

## Another Morpheus example

In [67]:
F:=FreeGroup(2);

<group with 2 generators>

In [68]:
F:=F/[F.1^2,F.2^6];

<group with 2 generators>

In [69]:
GQuotients(F,g);

[ [ f1, f2 ] -> [ (3,4)(5,8)(6,9)(7,10)(20,29)(21,30)(22,31)(23,32)(24,33)(25,34)(26,35)(27,36)(28,37)(38,65)(39,66)(40,67)(41,68)(42,69)(43,70)(44,71)(45,72)(46,73)(47,74)(48,75)(49,76)(50,77)(51,78)(52,79)(53,80)(54,81)(55,82)(56,83)(57,84)(58,85)(59,86)(60,87)(61,88)(62,89)(63,90)(64,91), (1,28,80,55,75,37)(2,79,73,78,64,76)(3,4,38,31,25,90)(5,33,23,39,42,65)(6,71,57)(7,84,54,72,13,52)(8,49,16,56,47,44)(9,14)(10,63,87,89,32,24)(11,86,58,61,35,26)(12,17,69,36,20,60)(15,40,88)(18,30,21,67,70,41)(19,81,46,77,91,74)(22,34)(27,50,29)(43,48,59,68,53,85)(45,62)(66,83) ], [ f1, f2 ] -> [ (2,3)(5,7)(6,8)(9,10)(11,30)(12,31)(13,29)(14,33)(15,37)(16,36)(17,34)(18,35)(19,32)(38,79)(39,81)(40,80)(41,75)(42,76)(43,82)(44,78)(45,77)(46,74)(47,62)(48,61)(49,63)(50,56)(51,57)(52,60)(53,59)(54,64)(55,58)(65,87)(66,91)(67,86)(68,88)(69,90)(70,85)(71,84)(72,83)(73,89), (1,28,80,55,75,37)(2,79,73,78,64,76)(3,4,38,31,25,90)(5,33,23,39,42,65)(6,71,57)(7,84,54,72,13,52)(8,49,16,56,47,44)(9,14)(10,63,87,89,

* So $U_3(3)$ is (2,6)-generated in two distinct ways
* Presented as homomorphisms — easy to recover the generators if you want them
* Other Morpheus functions: `AllHomomorphisms`, `AutomorphismGroup`, `IsomorphicSubgroups`
* A powerful tool for many purposes

## Anecdote: Extreme Searching (by Steve Linton)

* Looking for 2,3,7 triples in a a permutation group $G$ so big that main memory could only hold two permutations
    * expecting to have to check millions of cases
        * on a 16MHz CPU shared with the entire university
            * this was a while ago
* Know (from character table) all the conjugacy classes of elements order 2 and 3 and have representatives
* Fix T of order 2 and S of order 3 which generate $G$
    * run through all words W in ST and SST up to some length
    * trace points one by one through $(TW^{-1}SW)^7$
    * as soon as one does not get to the start you can discard that $W$
* Searches tens of thousands of cases for the cost of one permutation multiply

In [70]:
f := FreeGroup("a","b");

<group with 2 generators>

In [71]:
AssignGeneratorVariables(f);

#I  Assigned the global variables [ a, b ]


In [88]:
g := f/[a^2,b^3,(a*b)^7, Comm(a,b)^8];

<group with 2 generators>

In [89]:
Sum(Elements(g), Order);

66655

In [90]:
x := Random(g);

<object>

In [91]:
Print(x);

(a*b)^2*a*(b^-1*a*b*a*b^-1)^2*(b^-1*a^-1)^2*b*(a^-1*b*a^-1*b^-1)^2*(a^-1*b)^2*\
b*a*b^-1*(a*b^-1*a*b)^2*(a*b^-1)^2*(a*b)^2

In [75]:
g := f/[a^2,b^3,(a*b)^7, Comm(a,b)^8];

<group with 2 generators>

In [83]:
phi := IsomorphismPermGroup(g);

<object>

In [84]:
Print(phi);

GroupHomomorphismByImages( Group( [ a, b ] ), Group( 
[ ( 1, 2)( 3, 5)( 4, 6)( 7,11)( 8,12)( 9,13)(10,14)(16,20)(17,21)(18,22)
    (19,23)(25,29)(26,27)(28,30)(31,34)(32,35)(33,36)(37,41)(38,42)(39,43)
    (40,44)(45,50)(46,51)(48,52)(49,53)(54,56), 
  ( 2, 3, 4)( 5, 7, 8)( 6, 9,10)(11,15,14)(12,16,17)(13,18,19)(20,24,23)
    (21,25,26)(22,27,28)(29,31,32)(30,33,34)(35,37,38)(36,39,40)(41,45,46)
    (42,47,43)(44,48,49)(50,53,54)(51,55,52) ] ), [ a, b ], 
[ ( 1, 2)( 3, 5)( 4, 6)( 7,11)( 8,12)( 9,13)(10,14)(16,20)(17,21)(18,22)
    (19,23)(25,29)(26,27)(28,30)(31,34)(32,35)(33,36)(37,41)(38,42)(39,43)
    (40,44)(45,50)(46,51)(48,52)(49,53)(54,56), 
  ( 2, 3, 4)( 5, 7, 8)( 6, 9,10)(11,15,14)(12,16,17)(13,18,19)(20,24,23)
    (21,25,26)(22,27,28)(29,31,32)(30,33,34)(35,37,38)(36,39,40)(41,45,46)
    (42,47,43)(44,48,49)(50,53,54)(51,55,52) ] )

In [77]:
h := ImagesSource(phi);

<permutation group of size 10752 with 2 generators>

In [78]:
Sum(Elements(h), Order);

66655

In [79]:
x := Random(h);

(1,11,42,29,28,23,52)(2,5,40,7,10,12,49)(3,15,4,22,38,19,32)(6,54,45,13,46,48,37)(8,24,9,26,18,17,34)(14,56,51,20,25,30,43)(16,39,21,50,55,53,33)(27,41,47,44,31,36,35)

In [85]:
y := PreImagesRepresentative(phi,x);

<object>

In [86]:
Print(y);

b^-1*(a^-1*b^-1*a^-1*b)^3*(a^-1*b^-1)^2*(a*b)^2*a

## A Few Homomorphism Operations

In [93]:
g:=Group((1,2,3,4),(1,2),(5,6,7));

Group([ (1,2,3,4), (1,2), (5,6,7) ])

In [98]:
iso:=IsomorphismPcGroup(g);

<object>

In [99]:
h:=Image(iso);

<group of size 72 with 5 generators>

In [100]:
z:=Centre(h);

<group with 1 generators>

In [101]:
SetCentre(g,PreImage(iso,z));

In [102]:
cl:=ConjugacyClasses(h);

[ <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object>, <object> ]

In [103]:
ncl:=[];

[  ]

In [104]:
for c in cl do
    nc:=ConjugacyClass(g,PreImage(iso,Representative(c)));
    SetSize(nc,Size(c));
    SetStabilizerOfExternalSet(nc, PreImage(iso,StabilizerOfExternalSet(c)));
    Add(ncl,nc);
od; 

In [105]:
List(ncl,Size);

[ 1, 1, 6, 8, 3, 1, 6, 8, 3, 6, 6, 8, 3, 6, 6 ]

In [107]:
SetConjugacyClasses(g,ncl);

## Homorphisms in General

In [108]:
g := Group((1,2),(3,4),(5,6),(7,8),(9,10,11),(11,12,13));

Group([ (1,2), (3,4), (5,6), (7,8), (9,10,11), (11,12,13) ])

In [110]:
Number(g, x-> Order(x) mod 2 = 1); Size(g);

45

960

In [111]:
Orbits(g,MovedPoints(g));

[ [ 1, 2 ], [ 3, 4 ], [ 5, 6 ], [ 7, 8 ], [ 9, 10, 11, 12, 13 ] ]

In [112]:
phi := ActionHomomorphism(g,[1..8]);

<object>

In [114]:
Print(phi);

<action homomorphism>

In [113]:
h := ImagesSource(phi);

Group([ (1,2), (3,4), (5,6), (7,8) ])

In [116]:
odds := Filtered(h, x->Order(x) mod 2 = 1);

[ () ]

In [117]:
p := PreImagesSet(phi,odds);

[ (), (11,12,13), (11,13,12), (10,11)(12,13), (10,11,12), (10,11,13), (10,12,11), (10,12,13), (10,12)(11,13), (10,13,11), (10,13,12), (10,13)(11,12), (9,10)(12,13), (9,10)(11,12), (9,10)(11,13), (9,10,11), (9,10,11,12,13), (9,10,11,13,12), (9,10,12,13,11), (9,10,12), (9,10,12,11,13), (9,10,13,12,11), (9,10,13), (9,10,13,11,12), (9,11,10), (9,11,12,13,10), (9,11,13,12,10), (9,11)(12,13), (9,11,12), (9,11,13), (9,11)(10,12), (9,11,10,12,13), (9,11,13,10,12), (9,11)(10,13), (9,11,10,13,12), (9,11,12,10,13), (9,12,13,11,10), (9,12,10), (9,12,11,13,10), (9,12,11), (9,12,13), (9,12)(11,13), (9,12,13,10,11), (9,12)(10,11), (9,12,10,11,13), (9,12,10,13,11), (9,12,11,10,13), (9,12)(10,13), (9,13,12,11,10), (9,13,10), (9,13,11,12,10), (9,13,11), (9,13,12), (9,13)(11,12), (9,13,12,10,11), (9,13)(10,11), (9,13,10,11,12), (9,13,10,12,11), (9,13,11,10,12), (9,13)(10,12) ]

In [119]:
odds := Filtered(h, x->Order(x) mod 2 = 1);; Length(odds);

1

In [120]:
p := PreImagesSet(phi,odds);

[ (), (11,12,13), (11,13,12), (10,11)(12,13), (10,11,12), (10,11,13), (10,12,11), (10,12,13), (10,12)(11,13), (10,13,11), (10,13,12), (10,13)(11,12), (9,10)(12,13), (9,10)(11,12), (9,10)(11,13), (9,10,11), (9,10,11,12,13), (9,10,11,13,12), (9,10,12,13,11), (9,10,12), (9,10,12,11,13), (9,10,13,12,11), (9,10,13), (9,10,13,11,12), (9,11,10), (9,11,12,13,10), (9,11,13,12,10), (9,11)(12,13), (9,11,12), (9,11,13), (9,11)(10,12), (9,11,10,12,13), (9,11,13,10,12), (9,11)(10,13), (9,11,10,13,12), (9,11,12,10,13), (9,12,13,11,10), (9,12,10), (9,12,11,13,10), (9,12,11), (9,12,13), (9,12)(11,13), (9,12,13,10,11), (9,12)(10,11), (9,12,10,11,13), (9,12,10,13,11), (9,12,11,10,13), (9,12)(10,13), (9,13,12,11,10), (9,13,10), (9,13,11,12,10), (9,13,11), (9,13,12), (9,13)(11,12), (9,13,12,10,11), (9,13)(10,11), (9,13,10,11,12), (9,13,10,12,11), (9,13,11,10,12), (9,13)(10,12) ]

In [122]:
Length(p); Number(p, x->Order(x) mod 2 = 1);

60

45